In [1]:
import os
import pyodbc
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential 
from keras.layers import LSTM
from keras.layers import Dense 
from keras.layers import Dropout
import mysql.connector
mydb=mysql.connector.connect(host='localhost', user='root', password='himadri', database='dbms')

mycursor=mydb.cursor()
df= pd.read_sql_query('''select * from googlestock''',mydb)
df.head()
training_set = df.iloc[:,1:2].values

print(training_set)
print(training_set.shape)

print("Dataframe Shape:" , df. shape)
features = ['Open', 'High', 'low', 'Volume']

#Scaling
scaler = MinMaxScaler()
scaler = MinMaxScaler(feature_range = (0,1))
scaled_training_set = scaler.fit_transform(training_set)

scaled_training_set

X_train = []
Y_train = []
k=int(input("Enter the Time-Step:"))
for i in range(k,7):
    X_train.append(scaled_training_set[i-1:i, 0]) #n: time-step and 7:lenght of the data
    Y_train.append(scaled_training_set[i, 0])
X_train = np.array (X_train)
Y_train = np.array(Y_train)
print(X_train.shape)
print(Y_train.shape)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

a=X_train.shape
print(a)

regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences= True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout (0.2))

regressor.add(LSTM(units = 50, return_sequences= True)) 
regressor.add(Dropout (0.2))

regressor.add(LSTM(units = 50, return_sequences= True))
regressor.add(Dropout (0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout (0.2))

regressor.add(Dense (units=1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
regressor.fit(X_train, Y_train, epochs=100, batch_size=32)

dataset_test= pd.read_sql_query('''select * from googlestock''',mydb)
actual_stock_price=dataset_test.iloc[:,1:2].values

dataset_total=pd.concat((df, dataset_test), axis = 0)
inputs = dataset_total [len(dataset_total)- len(dataset_test)-k:].values
inputs = inputs.reshape(-1,1)
inputs = scaler.transform(inputs)
X_test = []
for i in range(k,7):
    X_test.append(inputs[i-k:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

predicted_stock_price=regressor.predict(X_test)
predicted_stock_price=scaler.inverse_transform(predicted_stock_price)

plt.plot(actual_stock_price, color = 'red', label = 'Actual Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted Google Stock Price')
plt.title('GOOGLE STOCK PRICE PREDICTION')
plt.xlabel('Time')
plt.ylabel('GOOGLE STOCK PRICE')
plt.legend()

print("\nThe below graph shows the stock price prediction and actual price of google stocks!")
    

AttributeError: module 'mysql.connector' has no attribute 'connect'